<a href="https://colab.research.google.com/github/pratzz/Diabetes-Readmission-Prediction/blob/main/Data_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from pandas_profiling import ProfileReport
import os
import time

In [2]:
timestr = time.strftime("%Y%m%d-%H%M%S")
output_path = os.path.join("../output")
data_path = "../data"
data_exploration_path = os.path.join(output_path, "data_exploration")
processed_data_path = os.path.join(data_path, "processed_data")
if not os.path.exists(data_exploration_path):
    os.makedirs(data_exploration_path) 
if not os.path.exists(processed_data_path):
    os.makedirs(processed_data_path)

In [3]:
train_data = pd.read_csv(os.path.join(data_path, "raw_data/train.csv"))
test_data = pd.read_csv(os.path.join(data_path, "raw_data/test.csv"))

# train_data.head(10)

### Data Profiling & Analysis 


In [4]:
profile_report = ProfileReport(train_data, title="Pandas Profiling Report")
profile_report.to_widgets()

In [5]:
profile_report.to_file(os.path.join(data_exploration_path, "profile_report_1__%s.html"%timestr)) 

##### Combining train_data and test_data


In [6]:
train_data['data']='train'
test_data['data']='test'
data=pd.concat([train_data,test_data],0,sort=False)
# data.head(10)
# data.columns

##### Drop columns

<br> Name: high cardinality
<br> Property ID: high cardinality
<br> Age: missing % = 15, autocorrelation = Income Stability, all classes equal distribution
<br> Gender: missing % = 0.2, all classes equal distribution
<br> Income Stability: missing % = 5.6, autocorrelation = Age & Profession (included)
<br> Expense Type 2: autocorrelation = Expense Type 1 (included)
<br> Has Active Credit Card: missing % = 5.2, all classes equal distribution
<br> Property Location: all classes equal distribution
<br> Property Type: all classes equal distribution

In [7]:
drop_cols = ["Name", "Age", "Gender", "Income Stability", "Expense Type 2", "Has Active Credit Card", "Property ID", "Property Location", "Property Type"]
data = data.drop(drop_cols, axis=1)

In [8]:
data.shape

(50000, 16)

##### Missing value treatment

<br> Income (USD): impute median
<br> Current Loan Expenses (USD): impute median
<br> Dependents: impute 0
<br> Property Age: impute median
<br> Type of Employment: impute nan as "missing_val"
<br> Loan Sanction Amount (USD): drop missing
<br> Property Price: replace string (?) with median
<br> Co-Applicant: replace string (?) with median

In [9]:
data.describe(include = "object").transpose()
data['Property Price'] = data['Property Price'].replace('?',np.nan).astype(float)
data['Co-Applicant'] = data['Co-Applicant'].replace('?',np.nan).astype(float)
data['Co-Applicant'] = data['Co-Applicant'].replace(-999.,np.nan).astype(float)

impute_median_col = ["Income (USD)", "Current Loan Expenses (USD)", "Property Age", "Credit Score", "Property Price","Co-Applicant"]

for col in impute_median_col:
        data.loc[data[col].isnull(),col]=data.loc[data['data']=='train',col].median()
data["Type of Employment"].fillna("missing_val", inplace=True)
data["Dependents"].fillna("0", inplace=True)
# data.isnull().sum()

In [10]:
data.describe(include = "object").transpose()

,count,unique,top,freq
Customer ID,50000,50000,C-37190,1
Profession,50000,8,Working,28331
Type of Employment,50000,19,missing_val,11959
Location,50000,3,Semi-Urban,36026
Expense Type 1,50000,2,N,32030
Dependents,50000,12,2,22059
data,50000,2,train,30000


##### Create categories 


<br> steps
<br> 1. select cut-off frequency
<br> 2. create categories

In [11]:
# cat_cols = data.select_dtypes(['object']).columns
# cat_cols

# convert to dummies

cat_cols1 = ["Type of Employment", "Expense Type 1", "Profession"]

for col in cat_cols1:
    freqs=data[col].value_counts()
    k=freqs.index[freqs>2000][:-1]
    for cat in k:
        name=col+'_'+cat
        data[name]=(data[col]==cat).astype(int)
    del data[col]
    
# convert to dummies
data["Dependents"] = data["Dependents"].astype(str)
cat_cols2 = ["Dependents"]

for col in cat_cols2:
    freqs=data[col].value_counts()
    k=freqs.index[freqs>300][:-1]
    for cat in k:
        name=col+'_'+cat
        data[name]=(data[col]==cat).astype(int)
    del data[col]
    
cat_cols3 = ["Location"]
for col in cat_cols3:
    freqs=data[col].value_counts()
    k=freqs.index[freqs>3000][:-1]
    for cat in k:
        name=col+'_'+cat
        data[name]=(data[col]==cat).astype(int)
    del data[col]        
   
data.shape

(50000, 29)

In [12]:
data.describe(include = "object").transpose()

,count,unique,top,freq
Customer ID,50000,50000,C-37190,1
data,50000,2,train,30000


##### Re-splitting data to train_data and test_data

In [13]:
train_data=data[data['data']=='train']
del train_data['data']
test_data=data[data['data']=='test']
del test_data['data']
del test_data['Loan Sanction Amount (USD)']
train_data.shape

(30000, 28)

##### Outlier treatment

<br> Income (USD): 1.5 * IQR
<br> Current Loan Expenses (USD): 1.5 * IQR
<br> Property Age: 1.5 * IQR
<br> Loan Sanction Amount (USD): <0

In [14]:
cols = ['Income (USD)', 'Current Loan Expenses (USD)', 'Property Age']

Q1 = train_data[cols].quantile(0.25)
Q3 = train_data[cols].quantile(0.75)
IQR = Q3 - Q1

train_data = train_data[~((train_data[cols] < (Q1 - 1.5 * IQR)) |(train_data[cols] > (Q3 + 1.5 * IQR))).any(axis=1)]

train_data.drop(train_data[train_data["Loan Sanction Amount (USD)"] < 0].index, inplace=True)
# train_data.shape

##### Missing value treatment

<br> Loan Sanction Amount (USD): drop missing

In [15]:
train_data = train_data.dropna(subset=["Loan Sanction Amount (USD)"])
train_data.shape
# train_data.transpose()

(26831, 28)

##### Splitting data into train and validation datasets 


In [16]:
target='Loan Sanction Amount (USD)'

X = train_data.drop([target],1)
y = train_data[target]
# Divide the training data given to train and validation data frames
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

##### Saving data on disk

In [17]:
x_train.to_csv(os.path.join(processed_data_path, "x_train.csv"),index=False)
x_val.to_csv(os.path.join(processed_data_path, "x_val.csv"),index=False)
y_train.to_csv(os.path.join(processed_data_path, "y_train.csv"),index=False)
y_val.to_csv(os.path.join(processed_data_path, "y_val.csv"),index=False)
test_data.to_csv(os.path.join(processed_data_path, "x_test.csv"),index=False)

### Data Profiling & Analysis of Processed train data


In [18]:
profile_report2 = ProfileReport(x_train, title="Pandas Profiling Report")
profile_report2.to_widgets()

In [19]:
profile_report2.to_file(os.path.join(data_exploration_path, "profile_report_2__%s.html"%timestr)) 